In [109]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

url = "https://www.tnwb.uscourts.gov/Search/search.aspx"
params = {
    'zoom_query' : 'CAR',
    'zoom_per_page' : 999999
}
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}
response = requests.get(url, headers=headers, params=params)
doc = BeautifulSoup(response.text)

In [110]:
cases = doc.find_all(True, {'class':['result_block' , 'result_altblock']})

In [111]:
rows = []
# Want to get URL, name, category, additional details

for case in cases:
    row = {}
    row['url'] = case.find('a')['href']
    row['title'] = case.find(class_='result_title').find('a').text
    row['category'] = case.find(class_='category').text.strip()
    row['details'] = case.find(class_='infoline').text
    rows.append(row)
print(f"{len(rows)} cases found")

132 cases found


In [112]:
df=pd.json_normalize(rows)
df[['Terms', 'Size', 'URL']] = df['details'].str.split(' - ', expand=True)
df.head()

,url,title,category,details,Terms,Size,URL
0,https://www.tnwb.uscourts.gov/Opinions/jdl/pdf...,JDL: 04-24318 Jacquelline D. Black,[Judges' Opinions],Terms matched: 1 - 102k - URL: https://ww...,Terms matched: 1,102k,URL: https://www.tnwb.uscourts.gov/Opinions/j...
1,https://www.tnwb.uscourts.gov/Opinions/whb/pdf...,WHB: 95-26401 Mary Lucy Cooper,[Judges' Opinions],Terms matched: 1 - 27k - URL: https://www...,Terms matched: 1,27k,URL: https://www.tnwb.uscourts.gov/Opinions/w...
2,https://www.tnwb.uscourts.gov/Opinions/ghb/pdf...,GHB: 97-12368 Billy G. Woffard,[Judges' Opinions],Terms matched: 1 - 71k - URL: https://www...,Terms matched: 1,71k,URL: https://www.tnwb.uscourts.gov/Opinions/g...
3,https://www.tnwb.uscourts.gov/Opinions/jdl/pdf...,JDL: 97-30580 Mary Chrlis Hurst,[Judges' Opinions],Terms matched: 1 - 32k - URL: https://www...,Terms matched: 1,32k,URL: https://www.tnwb.uscourts.gov/Opinions/j...
4,https://www.tnwb.uscourts.gov/Opinions/mrh/pdf...,MRH: 20-20967 Jacob Braxton Herring 20-00094,[Judges' Opinions],Terms matched: 1 - 303k - URL: https://ww...,Terms matched: 1,303k,URL: https://www.tnwb.uscourts.gov/Opinions/m...


In [113]:
df.to_csv("CAR_cases.csv")

In [114]:
i = 0
for url in df['url']:
    title = df['title'][i]
    title = re.sub(r'[<>:"/\\|?*]', '', title)
    pdf = open(f"cases/{title}.pdf", 'wb')
    response = requests.get(url)
    print(f"Downloaded {url}")
    pdf.write(response.content)
    pdf.close()
    print(f"saved title.pdf")
    print('---------------')
    i+=1

Downloaded https://www.tnwb.uscourts.gov/Opinions/jdl/pdf/jdl20071024nn1.pdf#search=%22car%22
saved title.pdf
---------------
Downloaded https://www.tnwb.uscourts.gov/Opinions/whb/pdf/whb19950809xn1.pdf#search=%22car%22
saved title.pdf
---------------
Downloaded https://www.tnwb.uscourts.gov/Opinions/ghb/pdf/ghb19980812xn1.pdf#search=%22car%22
saved title.pdf
---------------
Downloaded https://www.tnwb.uscourts.gov/Opinions/jdl/pdf/jdl19970918pn1.pdf#search=%22car%22
saved title.pdf
---------------
Downloaded https://www.tnwb.uscourts.gov/Opinions/mrh/pdf/mrh20220615nn1.pdf#search=%22car%22
saved title.pdf
---------------
Downloaded https://www.tnwb.uscourts.gov/Opinions/ghb/pdf/ghb19970417pn1.pdf#search=%22car%22
saved title.pdf
---------------
Downloaded https://www.tnwb.uscourts.gov/Opinions/jdl/pdf/jdl20090325nn1.pdf#search=%22car%22
saved title.pdf
---------------
Downloaded https://www.tnwb.uscourts.gov/Opinions/ghb/pdf/ghb20040325nn1.pdf#search=%22car%22
saved title.pdf
--------

In [115]:
print(f"Complete! {i} files saved.")

Complete! 132 files saved.
